# Face Recognition using a Pytorch CNN - Using LFW 
LFW Dataset - https://www.kaggle.com/jessicali9530/lfw-dataset

In [2]:
import pandas as pd
import os
import numpy as np
from PIL import Image
from matplotlib.pyplot import imshow
import math
import copy as cp
######
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [13]:
class FacialS_CNN(nn.Module):

    def __init__(self, inchannels = 3):
        super(FacialS_CNN, self).__init__()
        # 3 input image channel then 6 ch then 9 and finnally 12, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(inchannels, inchannels * 2, 7)
        self.conv2 = nn.Conv2d(inchannels * 2, inchannels * 3, 7)
        self.conv3 = nn.Conv2d(inchannels * 3, inchannels * 4, 7)
        self.conv4 = nn.Conv2d(inchannels * 4, inchannels * 5, 7)
        
        # an affine operation: y = Wx + b
        # Remember after a conv operation nout = (nin+2*npadding-nfilter)+1
        # in this example
        # conv1 = 250-7+1 = 244 => Maxpool => 122
        # conv2 = 122-7+1 = 122 => maxpool => 58
        # conv3 = 58-7+1 = 52 => maxpool => 26
        # conv4 = 26-7+1 = 20 => maxpool => 10
        self.fc1 = nn.Linear(inchannels * 5 * 10 * 10, 256)  # 10*10 from image dimension
        self.fc2 = nn.Linear(256, 200)
        self.fc3 = nn.Linear(200, 128)

    def forward(self, x, list_case, training = False):
            x = F.max_pool2d(F.relu(self.conv1(x.float())), 2)
            x = F.max_pool2d(F.relu(self.conv2(x)), 2)
            x = F.max_pool2d(F.relu(self.conv3(x)), 2)
            x = F.max_pool2d(F.relu(self.conv4(x)), 2)
            x = x.view(-1, self.num_flat_features(x))
            x = F.relu(self.fc1(x))
            x = F.relu(self.fc2(x))
            x = self.fc3(x)
            answer1 = cp.copy(x)
            if training == True:
                #triplet similar
                x = list_case[name][0]
                x = F.max_pool2d(F.relu(self.conv1(x.float())), 2)
                x = F.max_pool2d(F.relu(self.conv2(x)), 2)
                x = F.max_pool2d(F.relu(self.conv3(x)), 2)
                x = F.max_pool2d(F.relu(self.conv4(x)), 2)
                x = x.view(-1, self.num_flat_features(x))
                x = F.relu(self.fc1(x))
                x = F.relu(self.fc2(x))
                x = self.fc3(x)
                answer2 = cp.copy(x)
                #triplet different
                x = list_case[name][1]
                x = F.max_pool2d(F.relu(self.conv1(x.float())), 2)
                x = F.max_pool2d(F.relu(self.conv2(x)), 2)
                x = F.max_pool2d(F.relu(self.conv3(x)), 2)
                x = F.max_pool2d(F.relu(self.conv4(x)), 2)
                x = x.view(-1, self.num_flat_features(x))
                x = F.relu(self.fc1(x))
                x = F.relu(self.fc2(x))
                x = self.fc3(x)
                answer3 = cp.copy(x)
                return answer1, [answer2, answer3]
            return answer1
            
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
myCNN = FacialS_CNN()
print(myCNN)

FacialS_CNN(
  (conv1): Conv2d(3, 6, kernel_size=(7, 7), stride=(1, 1))
  (conv2): Conv2d(6, 9, kernel_size=(7, 7), stride=(1, 1))
  (conv3): Conv2d(9, 12, kernel_size=(7, 7), stride=(1, 1))
  (conv4): Conv2d(12, 15, kernel_size=(7, 7), stride=(1, 1))
  (fc1): Linear(in_features=1500, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=128, bias=True)
)


In [8]:
def str_converter(num, sz):
    num = str(num)
    while len(num) < sz:
        num = "0" + num
    return num

In [9]:
data_folder = "./Data/Data/"
face_folder = "lfw-deepfunneled/lfw-deepfunneled/"
name_ds = pd.read_csv(data_folder + "lfw/lfw_allnames.csv")
name_list = name_ds.values.tolist()
name = "Ricardo_Lazo_fr"
name_list.append([name, 6])

In [11]:
solved_list = []
acum = 0
for i in name_list:
    if i[1] >= 3:
        solved_list.append(cp.copy(i))
        acum += i[1]
print("nombres totales:", len(solved_list), "imagenes totales:", acum)

nombres totales: 902 imagenes totales: 7612


In [15]:
alfa = 0.1
def myLoss(answer1, answer2):
    [similar, different] = answer2
    return max(((torch.sum((answer1 - similar) ** 2) - (torch.sum((answer1 - different) ** 2))) / 128 + alfa ), 0)

In [17]:
# loss
loss_fn = myLoss
# learning rate
learning_rate = 1e-4
# optimizer
optimizer = torch.optim.Adam(myCNN.parameters(), lr=learning_rate)

In [ ]:
for it in range(500):
    for i in range(len(solved_list)):
        name = solved_list[i][0]
        rival = solved_list[(i + 1) % 902][0]
        im = Image.open(data_folder + "lfw/" + face_folder + name + "/" + name + "_" + str_converter(1, 4) + ".jpg")
        temp_np = np.reshape(np.array(im), (1,3,250,250))
        x = Variable(torch.from_numpy(temp_np)) 
        list_case = []
        
        im = Image.open(data_folder + "lfw/" + face_folder + name + "/" + name + "_" + str_converter(2, 4) + ".jpg")
        temp_np = np.reshape(np.array(im), (1,3,250,250))
        s = Variable(torch.from_numpy(temp_np)) 
        
        im = Image.open(data_folder + "lfw/" + face_folder + name + "/" + name + "_" + str_converter(1, 4) + ".jpg")
        temp_np = np.reshape(np.array(im), (1,3,250,250))
        r = Variable(torch.from_numpy(temp_np)) 
        
        list_case.append(s)
        list_case.append(r)
        
        # Forward pass: compute predicted y by passing x to the model.
        answer1, answer2 = myCNN(x, list_case, training = True) #4*1
        # Compute and print loss.
        loss = loss_fn(answer1, answer2)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if it % 100 == 99:
        print("it: ", it + 1)

In [ ]:
db = []
for i in solved_list:
    name = i[0]
    im = Image.open(data_folder + "lfw/" + face_folder + name + "/" + name + "_" + str_converter(1, 4) + ".jpg")
    temp_np = np.reshape(np.array(im), (1,3,250,250))
    x = Variable(torch.from_numpy(temp_np)) 
    answer1 = myCNN(x, [])
    db.append([name, answer1])

In [ ]:
def Hdistance(a, b): #both tensors
    return (torch.sum((a - b) ** 2) / 128

In [ ]:
#just my case
name = "Ricardo_Lazo_fr"
im = Image.open(data_folder + "lfw/" + face_folder + name + "/" + name + "_" + str_converter(3, 4) + ".jpg")
temp_np = np.reshape(np.array(im), (1,3,250,250))
x = Variable(torch.from_numpy(temp_np)) 
answer1 = myCNN(x, [])

In [ ]:
similar = ""
distance = 10000000
for i in db:
    temp = Hdistance(answer1, i[1])
    if temp < distance:
        distance = temp
        similar = i[0]
print("Me paresco a:", similar)